In [1]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.optim as optim
import torch.nn as nn
from torchvision import models
from torch.utils.data import DataLoader
from PIL import Image
import pandas as pd
import os
print("import finish")

import finish


In [4]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnext101_32x8d(weights=models.ResNeXt101_32X8D_Weights.DEFAULT).to(device)
model = model.to(device)

Downloading: "https://download.pytorch.org/models/resnext101_32x8d-110c445d.pth" to /root/.cache/torch/hub/checkpoints/resnext101_32x8d-110c445d.pth
100%|██████████| 340M/340M [00:04<00:00, 84.3MB/s] 


In [ ]:
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 100).to(device)
print("fc setting finish")

In [ ]:
transform = {
    'train':transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val':transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
}
print("transform finish")

In [ ]:
from torchvision import datasets
from torch.utils.data import DataLoader, Dataset
import os
from PIL import Image


train_dir = "/kaggle/input/training-data/train/train"
val_dir = "/kaggle/input/training-data/val/val"
test_dir = "/kaggle/input/training-data/test/test"  

batch_size = 32
num_workers = 2  
pin_memory = True if torch.cuda.is_available() else False

train_dataset = datasets.ImageFolder(root=train_dir, transform=transform['train'])
train_loader = DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=pin_memory
)

val_dataset = datasets.ImageFolder(root=val_dir, transform=transform['val'])
val_loader = DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=pin_memory
)
idx_to_class = {v: k for k, v in train_dataset.class_to_idx.items()}

class TestDataset(Dataset):
    def __init__(self, test_dir, transform=None):
        self.test_dir = test_dir
        self.transform = transform
        self.image_names = os.listdir(test_dir) 
    
    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        image_name = self.image_names[idx]
        image_path = os.path.join(self.test_dir, image_name)
        image = Image.open(image_path).convert("RGB") 
        
        if self.transform:
            image = self.transform(image)
        
        image = image.to(device)

        return image_name, image 

test_dataset = TestDataset(test_dir=test_dir, transform=transform)
test_loader = DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=pin_memory
)

print("✅ Data is ready! Training, validation, and test datasets are loaded.")

In [ ]:
batch_size = 32
epochs = 10
learning_rate = 0.0001

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
model.train()
best_val_acc = 0
for epoch in range(epochs):
    
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        labels = torch.tensor([int(idx_to_class[label.item()])for label in labels]).to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)

    train_acc = 100.0 * correct / total
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}, Train Acc: {train_acc:.2f}%")

    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            labels = torch.tensor([int(idx_to_class[label.item()])for label in labels]).to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

    val_acc = 100.0 * correct / total
    print(f"Validation Loss: {val_loss/len(val_loader):.4f}, Validation Acc: {val_acc:.2f}%\n")
    
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_model.pth") 
        print(f"Best model saved with Validation Acc: {best_val_acc:.2f}%\n")
    
    model.train() 

print("Training complete!")

In [ ]:
torch.save(model.state_dict(), "finetuned_resnext101.pth")

In [ ]:
model.eval()

In [ ]:
import os
import pandas as pd
from PIL import Image
import torch

def predict_image(image_path, model, transform, topk=5):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)
    
    with torch.no_grad():
        outputs = model(image)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
        top_probs, top_labels = torch.topk(probabilities, topk)
    
    return top_labels.squeeze().cpu().tolist()

image_folder = "/kaggle/input/training-data/test/test"
output_csv = "./prediction.csv"

model = model.to(device)

predictions = []

for image_name in os.listdir(image_folder):
    image_path = os.path.join(image_folder, image_name)
    
    top5_labels = predict_image(image_path, model, transform['val'], topk=5) 
    
    pred_label = top5_labels[0]
    
    image_name_without_extension = os.path.splitext(image_name)[0]
    
    predictions.append([image_name_without_extension, pred_label])

pred_df = pd.DataFrame(predictions, columns=["image_name", "pred_label"])
pred_df.to_csv(output_csv, index=False)

print(f"Predictions saved to {output_csv}")